# Tarea 1. Función de correlación de galaxias luminosas rojas en SDSS

En este proyecto, usaremos el catálogo espectroscópico de galaxias CMASS de SDSS para calcular la función angular de correlación de galaxias luminosas rojas (Luminous Red Galaxies, LRG).

En adelante se describe más específicamente los pasos que deberán seguir. En términos generales, se espera que el reporte contenga (al menos) las siguientes secciones:

* Introducción
* Datos
* Resultados
* Discusión y/o Conclusiones

He estructurado las instrucciones (ver más abajo) de esta manera. El reporte no necesita estar estructurado exactamente como sugieren estas instrucciones, pero pueden ser considerados una guía. Además, al comienzo del reporte deberá haber un resumen (abstract), en el que se describe brevemente la motivación, qué se hizo, y qué resultados se obtuvieron, y al final una lista de las referencias usadas, las que deberán ser mencionadas en el texto principal según corresponda. Pueden seguir el ejemplo de cualquier revista de astronomía o cosmología para decidir el formato de las referencias tanto en el texto como en el listado final.

Este reporte podrá ser escrito en cualquier editor de su preferencia, y tanto el pdf (obligatorio) como los archivos fuente, así como todo el código fuente utilizado en el proyecto, deberán ser entregados en un repositorio `git`, hospedado en algún servidor de los que existen para este propósito. Los dos más conocidos son [github](https://github.com) y [bitbucket](https://bitbucket.org/), pero podrán usar cualquiera de su preferencia. A tener en cuenta:
* Este repositorio *no debe* incluir los datos utilizados; para esto asegúrense de configurar el archivo `.gitignore`.
* El código fuente debe estar documentado lo suficiente como para que cualquier lector pueda seguirlo.
* Los códigos deberán estar referenciados en el reporte según corresponda.
* El reporte puede estar escrito en castellano o en inglés. Se recomienda que lo escriban en inglés; correcciones de lenguaje no tendrán incidencia en la calificación, aunque el texto deberá ser fácilmente legible.
* No hay límites inferior ni superior en el número de páginas; lo importante es el contenido, no la cantidad. En ese sentido, menos es mejor (siempre que se entregue toda la información necesaria!).

La fecha de entrega de este trabajo es el **Viernes 29 de Mayo, a las 14:15 hrs.** Es decir, antes de empezar la clase ese día, descargaré cada uno de los repositorios a mi computador, de manera de congelar la versión a evaluar a aquélla que exista *en la rama principal* del repositorio a la hora de la entrega.

## 1. Datos

Lo primero es descargar los datos de SDSS. Para esto, necesitan una cuenta en [`CasJobs`](https://skyserver.sdss.org/casjobs/) y luego ejecutar una búsqueda con `SQL` (bajo la sección `Query`). Los criterios para seleccionar estas galaxias están descritos (por ejemplo) en [Anderson et al. (2014)](https://ui.adsabs.harvard.edu/abs/2014MNRAS.441...24A/abstract) (ecuaciones 7-14). Ustedes deberán agregar estos criterios, además de cualquier columna adicional que les parezca necesaria para el análisis, al siguiente ejemplo `SQL`:

```SQL
select
  p.objid, p.ra, p.dec, 
  p.dered_u as umag, p.dered_g as gmag, p.dered_r as rmag, p.dered_i as imag, p.dered_z as zmag,
  s.z_noqso as z
FROM PhotoObj AS p
  JOIN SpecObj AS s ON s.bestobjid = p.objid
WHERE 
  s.z_noqso > 0
  AND s.zWarning_noqso = 0
  AND s.class = 'GALAXY'
```

Las tres condiciones mostradas aseguran que seleccionan galaxias con redshifts válidos, y tendrán que argumentar qué significa cada columna (por ejemplo: por qué `z_noqso` en lugar de `z`?) y por qué están ahí estas condiciones. Al ejecutar la búsqueda en `CasJobs`, deben asegurarse de seleccionar `DR16` en `Context`, de manera de descargar los datos más recientes. El código de búsqueda deberá ser incluido en la entrega como parte del código fuente.

Pueden ver más detalles sobre la información disponible en la base de datos [aquí](https://skyserver.sdss.org/dr12/en/help/browser/browser.aspx), revisando las tablas en la sección `Views` (que son tablas simplificadas a partir de aquéllas descritas en `Tables`).

Asegúrense de describir los datos que están usando: algo sobre SDSS y el catálogo que usaron para buscar las galaxias; de qué se trata la selección que hicieron; qué son estas galaxias rojas luminosas y qué tienen de especial; cuáles son sus características observacionales; etc.

El reporte deberá incluir una revisión visual de los datos, mostrando y describiendo las características más relevantes.

## 2. Análisis

### 2.1 Mediciones

Deberán calcular la función de correlación de correlación angular de LRGs usando el estimador de Landy-Szalay (Landy & Szalay, 1993),

$$
w(r) = \frac{DD(r) - 2DR(r) + RR(r)}{RR(r)},
$$

donde $DD(r)$, $DR(r)$, y $RR(r)$ son el número de pares a una distancia $r$ entre pares de datos en el catálogo real, pares entre el catálogo real y aleatorio, y pares en el catálogo aleatorio, respectivamente (asegúrense de contar cada par de galaxias una sola vez). Tomen en cuenta que las coordenadas son esféricas, de manera que la distancia entre dos puntos, con coordenadas (RA,Dec) $(\alpha_1,\delta_1)$ y $(\alpha_2,\delta_2)$, está dada por

$$
d = \arccos \left[ \sin(\delta_1)\sin(\delta_2) + \cos(\delta_1)\cos(\delta_2)\cos(\alpha_1-\alpha_2) \right]
$$

Esta distancia debe ser convertida a una distancia transversal comóvil al redshift correspondiente.

**Recomendaciones importantes**: 
* Calcular las distancias tomará tiempo, seguramente más de un día-CPU (además del tiempo que les tome programarlo correctamente, por supuesto). Se recomienda fuertemente que avancen hasta este paso lo antes posible, para no quedarse sin tiempo de hacer el análisis hacia el final de la tarea.
* No guarden las distancias mismas, pues esto literalmente les llenaría el disco. En lugar de eso, vayan actualizando un histograma de las distancias a medida que las van calculando. Un consejo: vayan generando simuláneamente dos histogramas, uno con bins lineales y otro con bins logarítmicos, y usen bins muy finos, de manera que más adelante puedan disminuir la resolución según les parezca.
* Les recomiendo guardarlas en algún tipo de archivo binario, en lugar de ASCII.
* Asegúrense que el catálogo aleatorio sea suficientemente grande para minimizar la incerteza estadística. Noten que si el catálogo real y el catálogo aleatorio tienen el mismo tamaño, ambos contribuirán de igual manera a la incerteza estadística de las mediciones, y por lo tanto se recomienda que el catálogo aleatorio sea tan grande como la capacidad computacional lo permita.

### 2.2 Interpretación

En clase sólo desarrollamos la teoría lineal de formación de estructuras. En detalle, la función de correlación es la suma de dos componentes:

$$
\xi(r) = \xi_\mathrm{1h}(r) + \xi_\mathrm{2h}(r),
$$

donde 1h y 2h se refieren a las contribuciones de un halo y de dos halos; es decir, las contribuciones de pares de galaxias que residen dentro de un mismo halo y pares de galaxias en halos distintos. Como vimos, la contribución 2h es sencilla de modelar a partir de la función de correlación de materia:

$$
\xi_\mathrm{gg,2h}(r) = b_\mathrm{gal}^2\xi_\mathrm{mm}(r).
$$

La contribución 1h, en cambio, depende de complicados detalles que tienen que ver con los mecanismos de formación de galaxias y sólo se puede modelar fenomenológicamente a partir de resultados obtenidos en simulaciones numéricas, con una metodología conocida como *modelo de halo*. En este trabajo, simplemente deberán modelar la función de correlación de galaxias como una suma de dos leyes de potencias actuando en cada uno de estos regímenes. La escala de transición deberá ser determinada, idealmente como parámetro libre pero de ser necesario es posible elegirla a mano (en este caso, no olviden justificar la elección). Sin embargo, deberá incluirse una discusión sobre el modelo de halo: de qué se trata, su utilidad para cosmología observacional (incluida su aplicación a otros observables, como lentes gravitacionales), limitaciones, etc. Si alguien se atreve, puede usar [HMFCalc](http://hmf.icrar.org/) (u otra herramienta) para obtener el espectro de potencias de materia y tratar de inferir el valor de $b_\mathrm{gal}$.

## 3. Discusión

Esta sección se refiere a aspectos que van más allá del trabajo mismo que se ha realizado, y muchas veces es absorbida en las otras secciones según decidan describir el análisis los autores.

### 3.1 Limitaciones de las mediciones

*Qué aspectos de las mediciones podrían limitar el análisis?*

A este respecto podrán discutir, por ejemplo, si el número de galaxias es suficiente, si se excluyeron ciertas escalas en el análisis, si hubo quizá limitaciones computacionales, etc. Deberán describir qué efecto tienen (y qué son) las colisiones de fibras en SDSS, y cualquier otro error sistemático que sean capaces de identificar, o cuya ausencia no se pueda establecer con los datos actuales.

### 3.2 Limitaciones del análisis

*Qué aspectos del análisis podrían limitar la interpretación de los resultados?*

Por ejemplo, la discusión sobre el modelo de halo ya mencionada, además de la importancia de una matriz de covarianza para mediciones de la función de correlación.

## 4. Conclusiones

Las conclusiones deben incluir un resumen del trabajo realizado, además de comentar sobre las posibilidades de expandir el trabajo a futuro, ya sea con datos nuevos o mejorando el análisis.